### ERA5-Land hourly data from 1950 to present

This Jupyter notebook will demonstrate how to access and visualise data from the Copernicus Climate Data Store (CDS) dataset **ERA5-Land hourly data from 1950 to present**. It will also show how the dataset's overview image and thumbnail image were generated.

For this demonstration, we will be using the Python package `earthkit` as a one-stop-shop for CDS data access, processing and visulisation. For more information on `earthkit`, including installation instructions, see its [GitHub Repository](https://github.com/ecmwf/earthkit).

In [ ]:
import earthkit

#### Retrieving CDS data

Data from the CDS can be accessed through earthkit via the `earthkit.data.from_source()` function. The first argument should be the string `'cds'`, followed by the dataset name (slug), and finally a dictionary containing the contents of your request.

In [ ]:
temperature = earthkit.data.from_source(
    'cds',
    'derived-era5-land-daily-statistics',
    {
        'variable': ['2m_temperature'],
        'year': '2023',
        'month': '01',
        'day': ['01'],
        'daily_statistic': 'daily_mean',
        'time_zone': 'utc+00:00',
        'frequency': '1_hourly',
        'download_format': 'unarchived'
    },
)

#### Generating a simple image

This section will demonstrate how the overview image for this dataset was generated.

To start with, we need to define some *styles* for plotting our variables. For 2m temperature, we want a shaded contour style using the *turbo* colour palette from [matplotlib](https://matplotlib.org/stable/gallery/color/colormap_reference.html). Styles can also have *units* associated with them, so that any data plotted using that style will be converted to use the style's units.

In [ ]:
temperature_style = earthkit.maps.styles.Contour(
    colors="turbo",
    levels=range(-40, 41, 2),
    extend="both",
    units="celsius",
)

Now we can produce our plot, using the styles we just defined. We are using *cartopy* to specify a *Nearside Perspective* projection for our plot.

In [ ]:
import cartopy.crs as ccrs
import earthkit.maps

earthkit.maps.schema.figsize = (4, 4)

crs = ccrs.NearsidePerspective(central_latitude=50)
chart = earthkit.plots.Map(crs=crs)

chart.plot(temperature, style=temperature_style, transform_first=False)

chart.coastlines(resolution="high")
chart.stock_img()

chart.legend(location="bottom")

# For the title, we can take advantage of earthkit-maps metadata
# detection, which allows us to use format keys like 'variable_name'
# and 'time', which are then populated from the metadata of the data
# that's been plotted.
chart.title(
    "ERA5-Land Daily statistics {variable_name}\n"
    "{time:%-d %B %Y} at {time:%H:%M} UTC",
    fontsize=11,
)

chart.save("overview-detail.png")

In [ ]:
earthkit.maps.schema.figsize = (2, 2)
chart = earthkit.maps.Superplot(crs=crs)

chart.plot(temperature, style=temperature_style, transform_first=False)

chart.coastlines(resolution="high")
chart.stock_img()

chart.save("overview.png")